# RunPod GPU 관리 노트북

REST API 직접 호출 방식 (serverless-mlops 패턴).
GPUType enum으로 타입 안전하게 GPU를 지정하고, pandas DataFrame으로 Pod 목록을 확인한다.

| GPU       | VRAM | ~$/hr | 용도          |
|-----------|------|------|-------------|
| RTX 4090  | 24GB | 0.39 | 2B 모델 학습/추론 |
| L40S      | 48GB | 0.74 | 7B 모델 서빙    |
| A100 80GB | 80GB | 1.64 | 대형 모델       |

In [1]:
import os, sys
from dotenv import load_dotenv

load_dotenv()
sys.path.insert(0, os.path.abspath(".."))

from utils.runpod_client import *

print("RunPod REST 클라이언트 로드 완료")

RunPod REST 클라이언트 로드 완료


In [ ]:
# 셀 2: Pod 목록 조회 → DataFrame
import pandas as pd

pod_list = pods()
if pod_list:
    df = pd.DataFrame(pod_list)
    display(df)
else:
    print("실행 중인 Pod이 없습니다.")

In [ ]:
# 셀 3: Pod 생성 (GPUType enum 사용)
pod_id = create(
    name="my-workspace",
    gpu_id=GPUType.NVIDIA_GEFORCE_RTX_4090,
    gpu_count=1,
    volume=50,
    image_name="runpod/pytorch:2.4.0-py3.11-cuda12.4.1-devel-ubuntu22.04",
)
print(f"생성된 Pod ID: {pod_id}")

In [ ]:
# 셀 4: 특정 Pod 상태 조회
# pod_id = "POD_ID"
pod_info = pod(pod_id)
pod_info

In [ ]:
# 셀 5: Pod 삭제
# pod_id = "POD_ID"
result = delete("6bo9b41ss21l6t")
print(f"삭제 결과: {result}")

In [ ]:
# 셀 6: VLM 서빙 Pod 생성
vlm_pod_id = create_from_template(
    name="qwen3-vl-4b-serving",
    gpu_id=GPUType.NVIDIA_GEFORCE_RTX_5080,
)
print(f"VLM Pod ID: {vlm_pod_id}")

In [2]:
# 셀 7: vLLM 연결 테스트
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

VLLM_BASE_URL = "https://api.runpod.ai/v2/qdwwc4uin801j6/openai/v1" # 예: http://your-pod-ip:8000/v1
MODEL_NAME = "qwen/qwen3-vl-4b-thinking"

if VLLM_BASE_URL != "여기에_URL을_입력하세요":
    llm = ChatOpenAI(
        base_url=VLLM_BASE_URL,
        model=MODEL_NAME,
        temperature=0,
        api_key=os.getenv("RUNPOD_API_KEY")
    )

    response = llm.invoke([
        HumanMessage(content="안녕! 너는 어떤 모델이니? 도구 호출이 가능하니?")
    ])
    print("\n[응답 결과]")
    print(response.content)
else:
    print("VLLM_BASE_URL을 입력해 주세요.")


[응답 결과]


안녕하세요! 저는 알리바바 그룹 산하 Tongyi 실험실에서 개발한 초대규모 언어 모델인 **Qwen**입니다. 

### 주요 기능:
- **질문 답변**: 다양한 주제에 대한 정보를 제공하거나 논리적 추론을 도와드립니다.
- **텍스트 생성**: 이야기, 공문서, 시나리오, 이메일 등 다양한 형식의 텍스트를 생성할 수 있습니다.
- **코드 작성**: 여러 프로그래밍 언어(예: Python, JavaScript, Java 등)를 지원하며, 문제 해결을 도와줍니다.
- **다국어 지원**: 한국어, 영어, 중국어, 일본어 등 100개 이상의 언어를 지원합니다.

### 도구 호출 가능 여부:
저는 **실시간 외부 도구나 API에 직접 접근할 수 없습니다**. 하지만, 도구 호출이 필요한 작업을 도와드릴 수 있습니다. 예를 들어:
- 특정 API를 사용해 데이터를 가져올 때, **코드 예시**를 제공해 주실 수 있습니다.
- 도구의 사용 방법을 설명해 주실 수 있습니다.
- 도구와의 연동을 위한 **구체적인 단계**를 안내해 드릴 수 있습니다.

예를 들어, "Google Maps API를 사용해 위치 정보를 검색하는 방법"에 대해 궁금한 점이 있으시면, 제가 코드 예시나 단계를 알려드릴 수 있습니다. 

혹시 구체적인 도구나 작업이 있으신가요? 함께 도와드리겠습니다! 😊
